# Actividad NLP 2

Mariana Castro Payns

Marcela Ibarra Mora

María de los Ángeles Arista Huerta

Eduardo Rodríguez Gil

Manolo Ramírez Pintor

Para subir el accuracy del modelo, se modificaron el numero de layers, neuronas y se cambio la arquitectura RNN por una GRU

In [1]:
import numpy as np
#PyTorch libraries
import torch


c:\Users\maria\anaconda3\envs\tf-gpu1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from torchtext.datasets import AG_NEWS
from torch import nn
from torch.nn import functional as F


In [3]:

# Dataset and dataloader
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset


In [4]:
torch.cuda.is_available()

True

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [6]:
train_dataset, test_dataset = AG_NEWS()

In [7]:
next(iter(train_dataset))

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

In [8]:
train_data, test_data = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [9]:
tokeniser = get_tokenizer('basic_english')

In [10]:
def yield_tokens(data):
    for _, text in data:
        yield tokeniser(text)

In [11]:
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

In [12]:
len(vocab)

95811

In [13]:
for i in range(10):
    print(i, vocab.lookup_token(i))

0 <unk>
1 .
2 the
3 ,
4 to
5 a
6 of
7 in
8 and
9 s


In [14]:
tokens = tokeniser('Welcome to TE3007B')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007b'] [3314, 4, 0]


In [15]:
NUM_TRAIN = int(len(train_data) * 0.9)
NUM_VAL = len(train_data) - NUM_TRAIN

In [16]:
NUM_VAL

12000

In [17]:
train_data, val_data = random_split(train_data, [NUM_TRAIN, NUM_VAL])


In [18]:
len(train_data), len(val_data), len(test_data)

(108000, 12000, 7600)

In [19]:
max_tokens = 50

In [20]:
def collate_batch(batch):
    y, x = list(zip(*batch)) #*batch para numero variable de elementos
    # create list with tokens 
    x = [vocab(tokeniser(text))for text in x]
    # padding
    x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]
    return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)-1 #para tener 0,1,2,3

In [21]:
labels = ['World', 'Sports', 'Business', 'Sci/Tech']
BATCH_SIZE = 1024

In [22]:
# Data loaders 
train_loader = DataLoader(train_data, 
                        batch_size=BATCH_SIZE, 
                        collate_fn= collate_batch, 
                        shuffle=True)

val_loader = DataLoader(val_data, 
                        batch_size=BATCH_SIZE, 
                        collate_fn= collate_batch, 
                        shuffle=True)

test_loader = DataLoader(test_data, 
                        batch_size=BATCH_SIZE, 
                        collate_fn= collate_batch, 
                        shuffle=True)

In [23]:
for i, (x,y) in enumerate(test_loader):
    print(i, x.shape, y.shape)

0 torch.Size([1024, 50]) torch.Size([1024])
1 torch.Size([1024, 50]) torch.Size([1024])
2 torch.Size([1024, 50]) torch.Size([1024])
3 torch.Size([1024, 50]) torch.Size([1024])
4 torch.Size([1024, 50]) torch.Size([1024])
5 torch.Size([1024, 50]) torch.Size([1024])
6 torch.Size([1024, 50]) torch.Size([1024])
7 torch.Size([432, 50]) torch.Size([432])


In [24]:
# Lets build our RNN
EMBEDDING_SIZE = 300
NEURONS = 300
LAYES = 2
NUM_CLASSES = 4

In [25]:
class RNN_model(nn.Module):
    def __init__(self, embed_size, hidden, layers, num_clases):
        super().__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                            embedding_dim=embed_size)

        self.rnn = nn.GRU(input_size=embed_size,
                        hidden_size=hidden,
                        num_layers = layers,
                        batch_first = True)

        self.fc = nn.Linear(in_features=hidden,
                            out_features=num_clases)
        
        self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        Vector_emb = self.embedding_layer(x)
        y, h = self.rnn(Vector_emb)
        return self.fc(y[:,-1])


In [26]:
rnn_model = RNN_model(EMBEDDING_SIZE, NEURONS, LAYES, NUM_CLASSES)

In [27]:
rnn_model

RNN_model(
  (embedding_layer): Embedding(95811, 300)
  (rnn): GRU(300, 300, num_layers=2, batch_first=True)
  (fc): Linear(in_features=300, out_features=4, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [28]:
def accuracy(model, loader):
    num_correct = 0
    num_total = 0
    model.eval()
    model = model.to(device=device)
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype = torch.long)
            y = y.to(device=device, dtype = torch.long)
            scores = model(x)
            _, preds = scores.max(dim=1)
            num_correct += (preds == y).sum()
            num_total += preds.size(0)
        acc = float(num_correct)/num_total
        return acc

In [29]:
def train(model, optimizer, epochs = 100):
    model = model.to(device=device)
    for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
            model.train()
            x = x.to(device=device, dtype=torch.long)
            y = y.to(device=device, dtype=torch.long)
            # Run model
            score = model(x)
            # compute cost
            cost = F.cross_entropy(input=score, target=y)
            # Fisrt gradient
            optimizer.zero_grad()
            # Compute gradeint
            cost.backward()
            # Update parameters
            optimizer.step()
        acc = accuracy(model, val_loader)
        print(f'Epoch {epoch}, costo {cost.item():.4f}, val_acc {acc:.4f}')

In [30]:
epochs = 15
lr = 0.0001

rnn_model = RNN_model(EMBEDDING_SIZE, NEURONS, LAYES, NUM_CLASSES)
optimizer = torch.optim.Adam(rnn_model.parameters(), lr=lr)

In [31]:
train(rnn_model, optimizer, epochs)

Epoch 0, costo 1.2854, val_acc 0.4064
Epoch 1, costo 0.6514, val_acc 0.7508
Epoch 2, costo 0.4784, val_acc 0.8413
Epoch 3, costo 0.3608, val_acc 0.8710
Epoch 4, costo 0.2831, val_acc 0.8787
Epoch 5, costo 0.3638, val_acc 0.8877
Epoch 6, costo 0.3170, val_acc 0.8903
Epoch 7, costo 0.2771, val_acc 0.8953
Epoch 8, costo 0.2554, val_acc 0.8976
Epoch 9, costo 0.2453, val_acc 0.8989
Epoch 10, costo 0.1967, val_acc 0.8964
Epoch 11, costo 0.2260, val_acc 0.8998
Epoch 12, costo 0.1260, val_acc 0.8992
Epoch 13, costo 0.1617, val_acc 0.9042
Epoch 14, costo 0.1267, val_acc 0.9008
